## Dataset COCO — Common Objects in Context

Прежде чем говорить о способах решения этих задач, надо разобраться с форматами входных данных. Сделаем это на примере датасета [COCO 🛠️[doc]](https://cocodataset.org/).

COCO — один из наиболее популярных датасатов, содержащий данные для сегментации и детектирования. Он содержит более трёхсот тысяч изображений, большая часть из которых размечена и содержит следующую информацию:
- категории;
- маски;
- ограничивающие боксы (*bounding boxes*);
- описания (*captions*);
- ключевые точки (*keypoints*);
- и многое другое.

Формат разметки изображений, использованный в этом датасете, нередко используется и в других наборах данных. Как правило, он упоминается просто как "COCO format".

Загрузим датасет:

In [ ]:
# !wget -qN "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"
!wget -qN "https://edunet.kea.su/repo/EduNet-web_dependencies/datasets/annotations_trainval2017.zip"
!unzip -qn annotations_trainval2017.zip

Для работы с датасетом используется пакет `pycocotools`.

[[blog] ✏️ Как создать свой COCO датасет с нуля](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch)

In [ ]:
from pycocotools.coco import COCO

coco = COCO("annotations/instances_val2017.json")

Рассмотрим формат аннотаций на примере одной записи:

In [ ]:
catIds = coco.getCatIds(catNms=["cat"])  #  Find category ID by tag
print("class ID(cat) =", catIds)

imgIds = coco.getImgIds(catIds=catIds)  # Filtering dataset by category ID
print("All images: %i" % len(imgIds))

Рассмотрим метаданные для первого изображения из категории:

In [ ]:
img_list = coco.loadImgs(imgIds[0])  # 1 example
img_metadata = img_list[0]
img_metadata

Посмотрим на изображение.

In [ ]:
import requests
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO


def coco2pil(url):
    response = requests.get(url)
    return Image.open(BytesIO(response.content))


I = coco2pil(img_metadata["coco_url"])
plt.axis("off")
plt.imshow(I)
plt.show()

### Категории в COCO

Давайте посмотрим на категории в датасете.

In [ ]:
cats = coco.loadCats(coco.getCatIds())  # loading categories
num2cat = {}
print("COCO categories: ")

iterator = iter(cats)
cat = next(iterator)
for i in range(0, 91):
    if i == cat["id"]:
        num2cat[cat["id"]] = cat["name"]
        name = cat["name"]
        if i < 90:
            cat = next(iterator)
    else:
        name = "---"

    print(f"{i:2}. {name:20}", end="")

    if not i % 6:
        print("\n")

Категория **0** используется для обозначения класса фона. Некоторые номера категорий не заняты.

Также существуют надкатегории.

In [ ]:
print(f"categories[2]: {cats[2]}")
print(f"categories[3]: {cats[3]}")

nms = set([cat["supercategory"] for cat in cats])
print("COCO supercategories: \n{}".format("\t".join(nms)))

### Разметка данных

Помимо метаданных нам доступна [разметка 🛠️[doc]](https://cocodataset.org/#format-data). Давайте её загрузим и отобразим.

In [ ]:
annIds = coco.getAnnIds(imgIds=img_metadata["id"])
anns = coco.loadAnns(annIds)

plt.imshow(I)
plt.axis("off")
coco.showAnns(anns)
plt.show()

На изображении можно увидеть разметку пикселей изображения по классам. То есть пиксели из объектов, относящихся к интересующим классам, приписываются к классу этого объекта. К примеру, можно увидеть объекты двух классов: "cat" и "keyboard".

Давайте теперь посмотрим, из чего состоит разметка.

In [ ]:
def dump_anns(anns):
    for i, a in enumerate(anns):
        print(f"\n#{i}")
        for k in a.keys():
            if k == "category_id" and num2cat.get(a[k], None):
                print(k, ": ", a[k], num2cat[a[k]])  # Show cat. name
            else:
                print(k, ": ", a[k])


dump_anns(anns)

Заметим, что аннотация изображения может состоять из описаний нескольких объектов, каждое из которых содержит следующую информацию:
* `segmentation` — последовательность пар чисел ($x$, $y$) — координат каждой из вершин "оболочки" объекта;
* `area` — площадь объекта;
* `iscrowd` — несколько объектов, например, толпа людей, в этом случае информация о границах объекта (маска) хранится в формате [RLE 📚[wiki]](https://en.wikipedia.org/wiki/Run-length_encoding);
* `image_id` — идентификатор изображения, к которому принадлежит описываемый объект;
* `bbox` — ограничивающий прямоугольник,*будет рассмотрен далее в ходе лекции*;
* `category_id` — идентификатор категории, к которой относится данный объект;
* `id` — идентификатор самого объекта.

Попробуем посмотреть на пример, в котором `iscrowd = True`.

In [ ]:
catIds = coco.getCatIds(catNms=["people"])
annIds = coco.getAnnIds(catIds=catIds, iscrowd=True)
anns = coco.loadAnns(annIds[0:1])

dump_anns(anns)
img = coco.loadImgs(anns[0]["image_id"])[0]
I = coco2pil(img["coco_url"])
plt.imshow(I)
coco.showAnns(anns)  # People in the stands
seg = anns[0]["segmentation"]
print("Counts", len(seg["counts"]))
print("Size", seg["size"])
plt.axis("off")
plt.show()

[[video] 📺 COCO Dataset Format — Complete Walkthrough](https://www.youtube.com/watch?v=h6s61a_pqfM)

Используя методы из `pycocotools`, можно  преобразовать набор вершин "оболочки" сегментируемого объекта в более удобный, но менее компактный вид — маску объекта.

In [ ]:
import numpy as np

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)
msk = np.zeros(seg["size"])

fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(10, 10))


i = 0
for row in range(4):
    for col in range(4):
        ann = anns[i]
        msk = coco.annToMask(ann)
        ax[row, col].imshow(msk, cmap="gray")
        ax[row, col].set_title(num2cat[anns[i]["category_id"]])
        ax[row, col].axis("off")
        i += 1

plt.show()

В некоторых случаях попиксельная разметка изображения может быть избыточной. К примеру, если необходимо посчитать количество человек на изображении, то достаточно просто каким-то образом промаркировать каждого из них, после чего посчитать количество наших "отметок". Одним из вариантов маркировки является "обведение" объекта рамкой (bounding box), внутри которой он находится. Такая информация об объектах также сохранена в аннотациях формата COCO.

In [ ]:
from PIL import ImageDraw

annIds = coco.getAnnIds(imgIds=[448263])
anns = coco.loadAnns(annIds)
draw = ImageDraw.Draw(I)

colors = {1: "white", 40: "lime"}  # person - white, glove - lime
for ann in anns:
    x, y, width, heigth = ann["bbox"]  # bounding box here
    color = colors.get(ann["category_id"], None)
    if color:
        draw.rectangle((x, y, x + width, y + heigth), outline=color, width=2)
plt.imshow(I)
plt.show()